DSE CODING PROJECT


Importing Libraries

In [3]:
import pandas as pd
import geopandas as gpd 
import streamlit as st
import matplotlib.pyplot as plt
from data_loading import load_data  # This is the function defined in the first file
import numpy as np

ModuleNotFoundError: No module named 'data_loading'

In [ ]:
# Load data
data = load_data(r"C:\Users\greta\OneDrive\Desktop\GlobalLandTemperaturesByMajorCity.csv")

In [ ]:
# Calculate temperature range for each city
temp_ranges = data.groupby('City')['AverageTemperature'].agg(lambda x: x.max() - x.min())
high_variance_cities = temp_ranges[temp_ranges > 15].index  # Adjust threshold as needed


In [ ]:
# Title and instructions
st.title("Global Temperature Change Visualization Over Time")
st.write("Explore the change in temperatures over time and identify cities with large temperature ranges.")


In [ ]:
# Year slider
year = st.slider("Select Year", int(data['Year'].min()), int(data['Year'].max()), step=1)


In [ ]:
# Filter data for selected year
year_data = data[data['Year'] == year]


In [ ]:
# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(year_data, geometry=gpd.points_from_xy(year_data.Longitude, year_data.Latitude))
